In [86]:
import torch
import numpy as np
from time import time

In [87]:
def timed(func): 
    def wrapper_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs) 
        t2 = time() 
        print(f'{(t2-t1):.8f}s') 
        return result 
    return wrapper_func 

Investigate the difference in time for numpy and torch's multivariate normal random generator

In [88]:
mean_np = np.array([0.0, 0.0])
cov_np = np.array([[1.0, 0.0], [0.0, 1.0]])

@timed
def numpy_multivariate_to_torch():
    for i in range(10000):
        torch.from_numpy(np.random.multivariate_normal(mean_np, cov_np, 10000))


numpy_multivariate_to_torch()

3.10636926s


In [89]:
mean_torch = torch.from_numpy(mean_np)
cov_torch = torch.from_numpy(cov_np)

@timed
def torch_multivariate():
    for i in range(10000):
        torch.distributions.MultivariateNormal(mean_torch, cov_torch).sample((10000,))

torch_multivariate()

6.83399343s


In [90]:
@timed
def torch_from_gaussia():
    for i in range(10000):
        torch.mm(torch.randn(10000, 2, dtype=torch.float64), cov_torch) + mean_torch

torch_from_gaussia()

4.09628749s


Investigate the same but for GPU

In [91]:
device = 'cuda'
print("Name of GPU:", torch.cuda.get_device_name(0))

Name of GPU: NVIDIA GeForce RTX 3050


In [92]:
@timed
def numpy_multivariate_cuda():
    for i in range(10000):
        torch.from_numpy(np.random.multivariate_normal(mean_np, cov_np, 10000)).cuda()


numpy_multivariate_to_torch()

3.13738084s


In [97]:
@timed
def torch_multivariate_cuda():
    for i in range(10000):
        torch.distributions.MultivariateNormal(mean_torch, cov_torch).sample((10000,)).cuda()

torch_multivariate_cuda()

9.78748703s


In [94]:
cov_cuda = cov_torch.to(device)
mean_cuda = mean_torch.to(device)

@timed
def torch_from_gaussian_cuda():
    for i in range(10000 ):
        x = torch.randn(10000, 2, dtype=torch.float64, device=device)
        x = torch.mm(x, cov_cuda) + mean_cuda

torch_from_gaussian_cuda()

1.41732359s


In [96]:
@timed
def torch_from_gaussian_cuda():
    for i in range(10000 ):
        x = torch.randn(10000, 2, dtype=torch.float64, device=device)
        x = torch.mm(x, cov_cuda) + mean_cuda
        x.cpu()

torch_from_gaussian_cuda()

2.03228903s
